In [0]:
from pyspark.sql.types import *

people_schema = StructType([
  StructField('name',StringType(),False),
  StructField('age',IntegerType(),False),
  StructField('city',StringType(),False)
])

In [0]:
from pyspark.sql import functions as F
aa_dfw_df = spark.read.format("csv").options(Header=True).load("/FileStore/tables/AA_DFW_2017_Departures_Short-5.csv")
aa_dfw_df = aa_dfw_df.withColumn("airport",F.lower(aa_dfw_df["Destination Airport"]))
aa_dfw_df = aa_dfw_df.drop(aa_dfw_df["Destination Airport"])
aa_dfw_df.show()

+-----------------+-------------+-----------------------------+-------+
Date (MM/DD/YYYY)|Flight Number|Actual elapsed time (Minutes)|airport|
+-----------------+-------------+-----------------------------+-------+
 01/01/2017| 0005| 537| hnl|
 01/01/2017| 0007| 498| ogg|
 01/01/2017| 0037| 241| sfo|
 01/01/2017| 0043| 134| dtw|
 01/01/2017| 0051| 88| stl|
 01/01/2017| 0060| 149| mia|
 01/01/2017| 0071| 203| lax|
 01/01/2017| 0074| 76| mem|
 01/01/2017| 0081| 123| den|
 01/01/2017| 0089| 161| slc|
 01/01/2017| 0096| 84| stl|
 01/01/2017| 0103| 216| sjc|
 01/01/2017| 0119| 514| ogg|
 01/01/2017| 0123| 529| hnl|
 01/01/2017| 0126| 171| lga|
 01/01/2017| 0132| 188| ewr|
 01/01/2017| 0140| 231| sjc|
 01/01/2017| 0174| 145| rdu|
 01/01/2017| 0176| 184| bos|
 01/01/2017| 0190| 76| sat|
+-----------------+-------------+-----------------------------+-------+
only showing top 20 rows

In [0]:
df1 = spark.read.format("csv").options(Header=True).load("/FileStore/tables/AA_DFW_2016_Departures_Short.csv")
df2 = spark.read.format("csv").options(Header=True).load("/FileStore/tables/AA_DFW_2017_Departures_Short-5.csv")
df3 = df1.union(df2)
df3 = df3.withColumnRenamed("Date (MM/DD/YYYY)","Date")
df3 = df3.withColumnRenamed("Flight Number","Flight_Number")
df3 = df3.withColumnRenamed("Actual elapsed time (Minutes)","Actual_elapsed_time")
df3 = df3.withColumnRenamed("Destination Airport","Destination_Airport")
# df3.write.parquet("/tmp/output/AA_DFW_ALL.parquet")
df = spark.read.format("parquet").load("/tmp/output/AA_DFW_ALL.parquet")
print(df1.count())
print(df2.count())
print(df.count())

140604
139358
279962

In [0]:
flights_df = spark.read.format("parquet").load("/tmp/output/AA_DFW_ALL.parquet")
flights_df.createOrReplaceTempView("flights")
short_flights_df = spark.sql("SELECT * FROM flights WHERE Actual_elapsed_time < 100")

In [0]:
from pyspark.sql.functions import length
voter_df = spark.read.format("csv").options(Header=True).load("/FileStore/tables/DallasCouncilVoters.csv")
voter_df.select("VOTER_NAME").distinct().show()
voter_df = voter_df.filter("length(VOTER_NAME) > 0 and length(VOTER_NAME) < 20")
voter_df = voter_df.where(~voter_df["VOTER_NAME"].contains("_"))
voter_df.select("VOTER_NAME").distinct().show()

+-------------------+
 VOTER_NAME|
+-------------------+
 Tennell Atkins|
 Scott Griggs|
 Scott Griggs|
 Sandy Greyson|
Michael S. Rawlings|
 Kevin Felder|
 Adam Medrano|
 Mark Clayton|
 Sandy Greyson|
 Jennifer S. Gates|
 B. Adam McGough|
 Omar Narvaez|
 Philip T. Kingston|
 Rickey D. Callahan|
 Dwaine R. Caraway|
 Casey Thomas

Philip T. Kingston|
 Jennifer S. Gates|
Rickey D. Callahan|
Carolyn King Arnold|
+-------------------+
only showing top 20 rows

+-------------------+
 VOTER_NAME|
+-------------------+
 Tennell Atkins|
 Scott Griggs|
 Scott Griggs|
 Sandy Greyson|
Michael S. Rawlings|
 Kevin Felder|
 Adam Medrano|
 Mark Clayton|
 Sandy Greyson|
 Jennifer S. Gates|
 B. Adam McGough|
 Omar Narvaez|
 Philip T. Kingston|
 Rickey D. Callahan|
 Dwaine R. Caraway|
 Casey Thomas

Philip T. Kingston|
 Jennifer S. Gates|
Rickey D. Callahan|
Carolyn King Arnold|
+-------------------+
only showing top 20 rows

In [0]:
voter_df = voter_df.withColumn('splits', F.split(voter_df.VOTER_NAME, '\s+'))
voter_df = voter_df.withColumn('first_name', voter_df.splits.getItem(0))
voter_df = voter_df.withColumn('last_name', voter_df.splits.getItem(F.size('splits') - 1))
voter_df = voter_df.drop('splits') 
voter_df.show()

+----------+-------------+-------------------+----------+---------+
 DATE| TITLE| VOTER_NAME|first_name|last_name|
+----------+-------------+-------------------+----------+---------+
02/08/2017|Councilmember| Jennifer S. Gates| Jennifer| Gates|
02/08/2017|Councilmember| Philip T. Kingston| Philip| Kingston|
02/08/2017| Mayor|Michael S. Rawlings| Michael| Rawlings|
02/08/2017|Councilmember| Adam Medrano| Adam| Medrano|
02/08/2017|Councilmember| Casey Thomas
 Casey| |
02/08/2017|Councilmember|Carolyn King Arnold| Carolyn| Arnold|
02/08/2017|Councilmember| Scott Griggs| Scott| Griggs|
02/08/2017|Councilmember| B. Adam McGough| B.| McGough|
02/08/2017|Councilmember| Lee Kleinman| Lee| Kleinman|
02/08/2017|Councilmember| Sandy Greyson| Sandy| Greyson|
02/08/2017|Councilmember| Jennifer S. Gates| Jennifer| Gates|
02/08/2017|Councilmember| Philip T. Kingston| Philip| Kingston|
02/08/2017| Mayor|Michael S. Rawlings| Michael| Rawlings|
02/08/2017|Councilmember| Adam Medrano| Adam| Medrano|
02/08/2017|Councilmember| Casey Thomas
 Casey| |
02/08/2017|Councilmember|Carolyn King Arnold| Carolyn| Arnold|
02/08/2017|Councilmember| Rickey D. Callahan| Rickey| Callahan|
01/11/2017|Councilmember| Jennifer S. Gates| Jennifer| Gates|
04/25/2018|Councilmember| Sandy Greyson| Sandy| Greyson|
04/25/2018|Councilmember| Jennifer S. Gates| Jennifer| Gates|
+----------+-------------+-------------------+----------+---------+
only showing top 20 rows

In [0]:
voter_df = voter_df.withColumn('random_val',F.when(voter_df.TITLE == 'Councilmember',F.rand()))
voter_df.show()

+----------+-------------+-------------------+----------+---------+--------------------+
 DATE| TITLE| VOTER_NAME|first_name|last_name| random_val|
+----------+-------------+-------------------+----------+---------+--------------------+
02/08/2017|Councilmember| Jennifer S. Gates| Jennifer| Gates| 0.752326490664182|
02/08/2017|Councilmember| Philip T. Kingston| Philip| Kingston| 0.3810401305037232|
02/08/2017| Mayor|Michael S. Rawlings| Michael| Rawlings| null|
02/08/2017|Councilmember| Adam Medrano| Adam| Medrano| 0.13886873171985303|
02/08/2017|Councilmember| Casey Thomas
 Casey| | 0.716776377328094|
02/08/2017|Councilmember|Carolyn King Arnold| Carolyn| Arnold| 0.8618713263920562|
02/08/2017|Councilmember| Scott Griggs| Scott| Griggs|0.040523245549255016|
02/08/2017|Councilmember| B. Adam McGough| B.| McGough| 0.5747049977623729|
02/08/2017|Councilmember| Lee Kleinman| Lee| Kleinman| 0.3338966096814334|
02/08/2017|Councilmember| Sandy Greyson| Sandy| Greyson| 0.9593713320528775|
02/08/2017|Councilmember| Jennifer S. Gates| Jennifer| Gates| 0.2159938786080955|
02/08/2017|Councilmember| Philip T. Kingston| Philip| Kingston| 0.4552569421160807|
02/08/2017| Mayor|Michael S. Rawlings| Michael| Rawlings| null|
02/08/2017|Councilmember| Adam Medrano| Adam| Medrano| 0.14882508341955425|
02/08/2017|Councilmember| Casey Thomas
 Casey| | 0.9402115280836887|
02/08/2017|Councilmember|Carolyn King Arnold| Carolyn| Arnold| 0.5368372547587485|
02/08/2017|Councilmember| Rickey D. Callahan| Rickey| Callahan|0.044414792046656326|
01/11/2017|Councilmember| Jennifer S. Gates| Jennifer| Gates| 0.7897503008944339|
04/25/2018|Councilmember| Sandy Greyson| Sandy| Greyson| 0.8150425523442287|
04/25/2018|Councilmember| Jennifer S. Gates| Jennifer| Gates| 0.18039471486666903|
+----------+-------------+-------------------+----------+---------+--------------------+
only showing top 20 rows

In [0]:
voter_df = voter_df.withColumn('random_val',F.when(voter_df.TITLE == 'Councilmember', F.rand()).when(voter_df.TITLE == 'Mayor', 2).otherwise(0))
voter_df.show()
voter_df.filter(voter_df.random_val==0).show()

+----------+-------------+-------------------+----------+---------+--------------------+
 DATE| TITLE| VOTER_NAME|first_name|last_name| random_val|
+----------+-------------+-------------------+----------+---------+--------------------+
02/08/2017|Councilmember| Jennifer S. Gates| Jennifer| Gates| 0.9666646298377839|
02/08/2017|Councilmember| Philip T. Kingston| Philip| Kingston| 0.16073414074304948|
02/08/2017| Mayor|Michael S. Rawlings| Michael| Rawlings| 2.0|
02/08/2017|Councilmember| Adam Medrano| Adam| Medrano| 0.16573340124384928|
02/08/2017|Councilmember| Casey Thomas
 Casey| | 0.9457205842980827|
02/08/2017|Councilmember|Carolyn King Arnold| Carolyn| Arnold| 0.972136940855039|
02/08/2017|Councilmember| Scott Griggs| Scott| Griggs| 0.7436029976814827|
02/08/2017|Councilmember| B. Adam McGough| B.| McGough| 0.08671823904160858|
02/08/2017|Councilmember| Lee Kleinman| Lee| Kleinman| 0.1734144282750817|
02/08/2017|Councilmember| Sandy Greyson| Sandy| Greyson| 0.5826695043548707|
02/08/2017|Councilmember| Jennifer S. Gates| Jennifer| Gates| 0.9571989601088341|
02/08/2017|Councilmember| Philip T. Kingston| Philip| Kingston| 0.6786369674816879|
02/08/2017| Mayor|Michael S. Rawlings| Michael| Rawlings| 2.0|
02/08/2017|Councilmember| Adam Medrano| Adam| Medrano| 0.8745125563918936|
02/08/2017|Councilmember| Casey Thomas
 Casey| | 0.12788221702710212|
02/08/2017|Councilmember|Carolyn King Arnold| Carolyn| Arnold| 0.918861772540459|
02/08/2017|Councilmember| Rickey D. Callahan| Rickey| Callahan| 0.8384433136830328|
01/11/2017|Councilmember| Jennifer S. Gates| Jennifer| Gates|0.053975208150005005|
04/25/2018|Councilmember| Sandy Greyson| Sandy| Greyson| 0.07221063604212041|
04/25/2018|Councilmember| Jennifer S. Gates| Jennifer| Gates| 0.5756734673916242|
+----------+-------------+-------------------+----------+---------+--------------------+
only showing top 20 rows

+----------+--------------------+-----------------+----------+---------+----------+
 DATE| TITLE| VOTER_NAME|first_name|last_name|random_val|
+----------+--------------------+-----------------+----------+---------+----------+
04/25/2018|Deputy Mayor Pro Tem| Adam Medrano| Adam| Medrano| 0.0|
04/25/2018| Mayor Pro Tem|Dwaine R. Caraway| Dwaine| Caraway| 0.0|
06/20/2018|Deputy Mayor Pro Tem| Adam Medrano| Adam| Medrano| 0.0|
06/20/2018| Mayor Pro Tem|Dwaine R. Caraway| Dwaine| Caraway| 0.0|
06/20/2018|Deputy Mayor Pro Tem| Adam Medrano| Adam| Medrano| 0.0|
06/20/2018| Mayor Pro Tem|Dwaine R. Caraway| Dwaine| Caraway| 0.0|
08/15/2018|Deputy Mayor Pro Tem| Adam Medrano| Adam| Medrano| 0.0|
08/15/2018|Deputy Mayor Pro Tem| Adam Medrano| Adam| Medrano| 0.0|
09/18/2018|Deputy Mayor Pro Tem| Adam Medrano| Adam| Medrano| 0.0|
09/18/2018| Mayor Pro Tem| Casey Thomas
 Casey| | 0.0|
04/25/2018|Deputy Mayor Pro Tem| Adam Medrano| Adam| Medrano| 0.0|
04/25/2018| Mayor Pro Tem|Dwaine R. Caraway| Dwaine| Caraway| 0.0|
04/11/2018| Mayor Pro Tem|Dwaine R. Caraway| Dwaine| Caraway| 0.0|
04/11/2018|Deputy Mayor Pro Tem| Adam Medrano| Adam| Medrano| 0.0|
04/11/2018| Mayor Pro Tem|Dwaine R. Caraway| Dwaine| Caraway| 0.0|
04/11/2018|Deputy Mayor Pro Tem| Adam Medrano| Adam| Medrano| 0.0|
04/11/2018| Mayor Pro Tem|Dwaine R. Caraway| Dwaine| Caraway| 0.0|
06/13/2018|Deputy Mayor Pro Tem| Adam Medrano| Adam| Medrano| 0.0|
06/13/2018| Mayor Pro Tem|Dwaine R. Caraway| Dwaine| Caraway| 0.0|
04/11/2018|Deputy Mayor Pro Tem| Adam Medrano| Adam| Medrano| 0.0|
+----------+--------------------+-----------------+----------+---------+----------+
only showing top 20 rows

In [0]:
def getFirstAndMiddle(names):
  return ' '.join(names[:-1])
udfFirstAndMiddle = F.udf(getFirstAndMiddle, StringType())
voter_df = voter_df.withColumn('splits', F.split(voter_df.VOTER_NAME, '\s+'))
voter_df = voter_df.withColumn('first_and_middle_name', udfFirstAndMiddle(voter_df.splits))
voter_df = voter_df.drop(voter_df.splits)
voter_df.show()

+----------+-------------+-------------------+----------+---------+--------------------+---------------------+
 DATE| TITLE| VOTER_NAME|first_name|last_name| random_val|first_and_middle_name|
+----------+-------------+-------------------+----------+---------+--------------------+---------------------+
02/08/2017|Councilmember| Jennifer S. Gates| Jennifer| Gates| 0.9666646298377839| Jennifer S.|
02/08/2017|Councilmember| Philip T. Kingston| Philip| Kingston| 0.16073414074304948| Philip T.|
02/08/2017| Mayor|Michael S. Rawlings| Michael| Rawlings| 2.0| Michael S.|
02/08/2017|Councilmember| Adam Medrano| Adam| Medrano| 0.16573340124384928| Adam|
02/08/2017|Councilmember| Casey Thomas
 Casey| | 0.9457205842980827| Casey Thomas|
02/08/2017|Councilmember|Carolyn King Arnold| Carolyn| Arnold| 0.972136940855039| Carolyn King|
02/08/2017|Councilmember| Scott Griggs| Scott| Griggs| 0.7436029976814827| Scott|
02/08/2017|Councilmember| B. Adam McGough| B.| McGough| 0.08671823904160858| B. Adam|
02/08/2017|Councilmember| Lee Kleinman| Lee| Kleinman| 0.1734144282750817| Lee|
02/08/2017|Councilmember| Sandy Greyson| Sandy| Greyson| 0.5826695043548707| Sandy|
02/08/2017|Councilmember| Jennifer S. Gates| Jennifer| Gates| 0.9571989601088341| Jennifer S.|
02/08/2017|Councilmember| Philip T. Kingston| Philip| Kingston| 0.6786369674816879| Philip T.|
02/08/2017| Mayor|Michael S. Rawlings| Michael| Rawlings| 2.0| Michael S.|
02/08/2017|Councilmember| Adam Medrano| Adam| Medrano| 0.8745125563918936| Adam|
02/08/2017|Councilmember| Casey Thomas
 Casey| | 0.12788221702710212| Casey Thomas|
02/08/2017|Councilmember|Carolyn King Arnold| Carolyn| Arnold| 0.918861772540459| Carolyn King|
02/08/2017|Councilmember| Rickey D. Callahan| Rickey| Callahan| 0.8384433136830328| Rickey D.|
01/11/2017|Councilmember| Jennifer S. Gates| Jennifer| Gates|0.053975208150005005| Jennifer S.|
04/25/2018|Councilmember| Sandy Greyson| Sandy| Greyson| 0.07221063604212041| Sandy|
04/25/2018|Councilmember| Jennifer S. Gates| Jennifer| Gates| 0.5756734673916242| Jennifer S.|
+----------+-------------+-------------------+----------+---------+--------------------+---------------------+
only showing top 20 rows

In [0]:
voter_df = voter_df.select(voter_df.VOTER_NAME).distinct()
print(voter_df.count())
voter_df = voter_df.withColumn("ROW_ID",F.monotonically_increasing_id())
voter_df.orderBy(voter_df.ROW_ID.desc()).show()

27
+-------------------+------+
 VOTER_NAME|ROW_ID|
+-------------------+------+
 Lee Kleinman| 26|
 Erik Wilson| 25|
Carolyn King Arnold| 24|
Rickey D. Callahan| 23|
 Monica R. Alonzo| 22|
 Lee M. Kleinman| 21|
 Casey Thomas
 20|
 Jennifer S. Gates| 19|
Philip T. Kingston| 18|
 Casey Thomas
 17|
 Dwaine R. Caraway| 16|
 Rickey D. Callahan| 15|
 Philip T. Kingston| 14|
 Omar Narvaez| 13|
 B. Adam McGough| 12|
 Tiffinni A. Young| 11|
 Jennifer S. Gates| 10|
 Mark Clayton| 9|
 Sandy Greyson| 8|
 Mark Clayton| 7|
+-------------------+------+
only showing top 20 rows

In [0]:
print(voter_df.rdd.getNumPartitions())
voter_df = voter_df.withColumn("ROW_ID",F.monotonically_increasing_id())
voter_df.orderBy(voter_df.ROW_ID.desc()).show()

1
+-------------------+------+
 VOTER_NAME|ROW_ID|
+-------------------+------+
 Lee Kleinman| 26|
 Erik Wilson| 25|
Carolyn King Arnold| 24|
Rickey D. Callahan| 23|
 Monica R. Alonzo| 22|
 Lee M. Kleinman| 21|
 Casey Thomas
 20|
 Jennifer S. Gates| 19|
Philip T. Kingston| 18|
 Casey Thomas
 17|
 Dwaine R. Caraway| 16|
 Rickey D. Callahan| 15|
 Philip T. Kingston| 14|
 Omar Narvaez| 13|
 B. Adam McGough| 12|
 Tiffinni A. Young| 11|
 Jennifer S. Gates| 10|
 Mark Clayton| 9|
 Sandy Greyson| 8|
 Mark Clayton| 7|
+-------------------+------+
only showing top 20 rows

In [0]:
## dataframe not available

In [0]:
## dataframe not available
import time
start_time = time.time()
df = voter_df.distinct().cache()
print(df.count(), time.time()-start_time)

start_time = time.time()
print(df.count(), time.time()-start_time)

27 5.560304403305054
27 3.241671085357666

In [0]:
print("Is df cached?: %s" % df.is_cached)
df.unpersist()
print("Is df cached?: %s" % df.is_cached)

Is df cached?: True
Is df cached?: False

In [0]:
## dataframe not available
df1 = spark.read.csv("/FileStore/tables/DallasCouncilVoters.csv")
df2 = spark.read.csv("/FileStore/tables/DallasCouncilVotes_csv.gz")

start_time = time.time()
print(df1.count())
print(" Time to run for df1 %f" %(time.time()-start_time))

start_time = time.time()
print(df2
      .count())
print(" Time to run for df2 %f" %(time.time()-start_time))


44626
 Time to run for df1 0.398433
44626
 Time to run for df2 0.681353

In [0]:
app_name = spark.conf.get("spark.app.name")
driver_tcp_port = spark.conf.get("spark.driver.port")
num_partitions = spark.conf.get("spark.sql.shuffle.partitions")

print(app_name,driver_tcp_port,num_partitions)

Databricks Shell 38585 200

In [0]:
## dataframe not available
before = df.rdd.getNumPartitions()
spark.conf.set('spark.sql.shuffle.partitions', 500)
df1 = spark.read.csv("/FileStore/tables/DallasCouncilVotes_csv.gz").distinct()
after = df1.rdd.getNumPartitions()
print(before)
print(after)

1
13

In [0]:
airports_df = spark.read.csv("/FileStore/tables/airports.csv",header=True)
flights_df = spark.read.csv("/FileStore/tables/flights.csv",header=True)
normal_df = flights_df.join(airports_df,flights_df["dest"] == airports_df["faa"] )
normal_df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [dest#920], [faa#879], Inner, BuildRight, false
 :- Filter isnotnull(dest#920)
 : +- FileScan csv [year#909,month#910,day#911,dep_time#912,dep_delay#913,arr_time#914,arr_delay#915,carrier#916,tailnum#917,flight#918,origin#919,dest#920,air_time#921,distance#922,hour#923,minute#924] Batched: false, DataFilters: [isnotnull(dest#920)], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/flights.csv], PartitionFilters: [], PushedFilters: [IsNotNull(dest)], ReadSchema: struct<year:string,month:string,day:string,dep_time:string,dep_delay:string,arr_time:string,arr_d...
 +- BroadcastExchange HashedRelationBroadcastMode(ArrayBuffer(input[0, string, false]),false), [id=#2017]
 +- Filter isnotnull(faa#879)
 +- FileScan csv [faa#879,name#880,lat#881,lon#882,alt#883,tz#884,dst#885] Batched: false, DataFilters: [isnotnull(faa#879)], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/airports.csv], PartitionFilters: [], PushedFilters: [IsNotNull(faa)], ReadSchema: struct<faa:string,name:string,lat:string,lon:string,alt:string,tz:string,dst:string>

In [0]:
from pyspark.sql.functions import broadcast
broadcast_df = flights_df.join(broadcast(airports_df),flights_df["dest"]==airports_df["faa"])
broadcast_df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [dest#920], [faa#879], Inner, BuildRight, false
 :- Filter isnotnull(dest#920)
 : +- FileScan csv [year#909,month#910,day#911,dep_time#912,dep_delay#913,arr_time#914,arr_delay#915,carrier#916,tailnum#917,flight#918,origin#919,dest#920,air_time#921,distance#922,hour#923,minute#924] Batched: false, DataFilters: [isnotnull(dest#920)], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/flights.csv], PartitionFilters: [], PushedFilters: [IsNotNull(dest)], ReadSchema: struct<year:string,month:string,day:string,dep_time:string,dep_delay:string,arr_time:string,arr_d...
 +- BroadcastExchange HashedRelationBroadcastMode(ArrayBuffer(input[0, string, false]),false), [id=#2086]
 +- Filter isnotnull(faa#879)
 +- FileScan csv [faa#879,name#880,lat#881,lon#882,alt#883,tz#884,dst#885] Batched: false, DataFilters: [isnotnull(faa#879)], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/airports.csv], PartitionFilters: [], PushedFilters: [IsNotNull(faa)], ReadSchema: struct<faa:string,name:string,lat:string,lon:string,alt:string,tz:string,dst:string>

In [0]:
start_time = time.time()

normal_count = normal_df.count()
normal_duration = time.time() - start_time
start_time = time.time()

broadcast_count = broadcast_df.count()
broadcast_duration = time.time() - start_time

print("Normal count:\t\t%d\tduration: %f" % (normal_count, normal_duration))
print("Broadcast count:\t%d\tduration: %f" % (broadcast_count, broadcast_duration))

Normal count: 10000	duration: 1.000889
Broadcast count:	10000	duration: 0.671036

In [0]:
df = spark.read.csv("/FileStore/tables/airports.csv", header=True)
df = df.filter(df[3] > 0)

df = df.withColumn('id', F.monotonically_increasing_id())
df.write.json('output.json', mode='overwrite')

In [0]:
votes_df = spark.read.csv("/FileStore/tables/DallasCouncilVotes_csv.gz", sep ='|')
full_count= votes_df.count()
comment_count = votes_df.where(votes_df['_c0'].startswith('#')).count()

no_comments_df = spark.read.csv("/FileStore/tables/DallasCouncilVotes_csv.gz", sep ='|', comment ='#')
no_comments_count = no_comments_df.count()

print("Full count: %d\nComment count: %d\nRemaining count: %d" % (full_count, comment_count, no_comments_count))

+--------------------+
 _c0|
+--------------------+
DATE,AGENDA_ITEM_...|
02/08/2017,1,AGEN...|
02/08/2017,1,AGEN...|
02/08/2017,1,AGEN...|
02/08/2017,1,AGEN...|
02/08/2017,1,AGEN...|
02/08/2017,1,AGEN...|
02/08/2017,4,AGEN...|
02/08/2017,4,AGEN...|
02/08/2017,4,AGEN...|
02/08/2017,4,AGEN...|
02/08/2017,4,AGEN...|
02/08/2017,4,AGEN...|
02/08/2017,4,AGEN...|
02/08/2017,4,AGEN...|
02/08/2017,4,AGEN...|
02/08/2017,4,AGEN...|
02/08/2017,4,AGEN...|
01/11/2017,1,AGEN...|
04/25/2018,1,AGEN...|
+--------------------+
only showing top 20 rows

Full count: 44626
Comment count: 0
Remaining count: 44626

In [0]:
## Dataset not available
initial_count = votes_df.count()
tmp_fields = F.split(votes_df['_c0'], ',')
votes_df = votes_df.withColumn('colcount', F.size(tmp_fields))
df_filtered = votes_df.filter(~ (votes_df["colcount"] < 5))
final_count = df_filtered.count()
print("Initial count: %d\nFinal count: %d" % (initial_count, final_count))

Initial count: 44626
Final count: 44431

In [0]:
## Dataset not available
annotations_df = votes_df
split_cols = F.split(annotations_df["_c0"], ',')
split_df = annotations_df.withColumn('DATE', split_cols.getItem(0))
split_df = split_df.withColumn('AGENDA_ITEM_NUMBER', split_cols.getItem(1))
split_df = split_df.withColumn('ITEM_TYPE', split_cols.getItem(2))
split_df = split_df.withColumn('DISTRICT', split_cols.getItem(3))
split_df = split_df.withColumn('TITLE', split_cols.getItem(4))
split_df = split_df.withColumn('VOTER_NAME', split_cols.getItem(5))
split_df = split_df.withColumn('VOTE_CAST', split_cols.getItem(6))
split_df = split_df.withColumn('FINAL_ACTION_TAKEN', split_cols.getItem(7))
split_df = split_df.withColumn('AGENDA_ITEM_DESC', split_cols.getItem(8))
split_df = split_df.withColumn('AGENDA_ID', split_cols.getItem(9))
split_df = split_df.withColumn('VOTE_ID', split_cols.getItem(10))
split_df = split_df.withColumn('split_cols', split_cols)
split_df.show()

+--------------------+--------+----------+------------------+-----------+--------+-------------+-------------------+---------+------------------+--------------------+--------------------+--------------------+--------------------+
 _c0|colcount| DATE|AGENDA_ITEM_NUMBER| ITEM_TYPE|DISTRICT| TITLE| VOTER_NAME|VOTE_CAST|FINAL_ACTION_TAKEN| AGENDA_ITEM_DESC| AGENDA_ID| VOTE_ID| split_cols|
+--------------------+--------+----------+------------------+-----------+--------+-------------+-------------------+---------+------------------+--------------------+--------------------+--------------------+--------------------+
DATE,AGENDA_ITEM_...| 11| DATE|AGENDA_ITEM_NUMBER| ITEM_TYPE|DISTRICT| TITLE| VOTER NAME|VOTE CAST|FINAL ACTION TAKEN|AGENDA ITEM DESCR...| AGENDA_ID| VOTE_ID|[DATE, AGENDA_ITE...|
02/08/2017,1,AGEN...| 11|02/08/2017| 1| AGENDA| 13|Councilmember| Jennifer S. Gates| N/A| NO ACTION NEEDED| Call to Order| 020817__Special__1|020817__Special__...|[02/08/2017, 1, A...|
02/08/2017,1,AGEN...| 11|02/08/2017| 1| AGENDA| 14|Councilmember| Philip T. Kingston| N/A| NO ACTION NEEDED| Call to Order| 020817__Special__1|020817__Special__...|[02/08/2017, 1, A...|
02/08/2017,1,AGEN...| 11|02/08/2017| 1| AGENDA| 15| Mayor|Michael S. Rawlings| N/A| NO ACTION NEEDED| Call to Order| 020817__Special__1|020817__Special__...|[02/08/2017, 1, A...|
02/08/2017,1,AGEN...| 11|02/08/2017| 1| AGENDA| 2|Councilmember| Adam Medrano| N/A| NO ACTION NEEDED| Call to Order| 020817__Special__1|020817__Special__1_2|[02/08/2017, 1, A...|
02/08/2017,1,AGEN...| 12|02/08/2017| 1| AGENDA| 3|Councilmember| "Casey Thomas| II"| N/A| NO ACTION NEEDED| Call to Order| 020817__Special__1|[02/08/2017, 1, A...|
02/08/2017,1,AGEN...| 11|02/08/2017| 1| AGENDA| 4|Councilmember|Carolyn King Arnold| N/A| NO ACTION NEEDED| Call to Order| 020817__Special__1|020817__Special__1_4|[02/08/2017, 1, A...|
02/08/2017,4,AGEN...| 20|02/08/2017| 4|AGENDA DATE| 1|Councilmember| Scott Griggs| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[02/08/2017, 4, A...|
02/08/2017,4,AGEN...| 20|02/08/2017| 4|AGENDA DATE| 10|Councilmember| B. Adam McGough| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[02/08/2017, 4, A...|
02/08/2017,4,AGEN...| 20|02/08/2017| 4|AGENDA DATE| 11|Councilmember| Lee Kleinman| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[02/08/2017, 4, A...|
02/08/2017,4,AGEN...| 20|02/08/2017| 4|AGENDA DATE| 12|Councilmember| Sandy Greyson| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[02/08/2017, 4, A...|
02/08/2017,4,AGEN...| 20|02/08/2017| 4|AGENDA DATE| 13|Councilmember| Jennifer S. Gates| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[02/08/2017, 4, A...|
02/08/2017,4,AGEN...| 20|02/08/2017| 4|AGENDA DATE| 14|Councilmember| Philip T. Kingston| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[02/08/2017, 4, A...|
02/08/2017,4,AGEN...| 20|02/08/2017| 4|AGENDA DATE| 15| Mayor|Michael S. Rawlings| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[02/08/2017, 4, A...|
02/08/2017,4,AGEN...| 20|02/08/2017| 4|AGENDA DATE| 2|Councilmember| Adam Medrano| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[02/08/2017, 4, A...|
02/08/2017,4,AGEN...| 21|02/08/2017| 4|AGENDA DATE| 3|Councilmember| "Casey Thomas| II"| YES| APPROVED|"Authorize a thre...| calibration|[02/08/2017, 4, A...|
02/08/2017,4,AGEN...| 20|02/08/2017| 4|AGENDA DATE| 4|Councilmember|Carolyn King Arnold| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[02/08/2017, 4, A...|
02/08/2017,4,AGEN...| 20|02/08/2017| 4|AGENDA DATE| 5|Councilmember| Rickey D. Callahan| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[02/08/2017, 4, A...|
01/11/2017,1,AGEN...| 12|01/11/2017| 1|AGENDA DATE| 13|Councilmember| Jennifer S. Gates| YES| APPROVED|"Approval of Minu...| 2016 City Counci...| 011117__1|[01/11/2017, 1, A...|
04/25/2018,1,A

In [0]:
def retriever(cols, colcount):
  return cols[10:colcount]

udfRetriever = F.udf(retriever, ArrayType(StringType()))

split_df = split_df.withColumn('dog_list', udfRetriever(split_df.split_cols, split_df.colcount))
splited_df = split_df.drop('_c0').drop('split_cols').drop('colcount')
splited_df.show()

+----------+------------------+-----------+--------+-------------+-------------------+---------+------------------+--------------------+--------------------+--------------------+--------------------+
 DATE|AGENDA_ITEM_NUMBER| ITEM_TYPE|DISTRICT| TITLE| VOTER_NAME|VOTE_CAST|FINAL_ACTION_TAKEN| AGENDA_ITEM_DESC| AGENDA_ID| VOTE_ID| dog_list|
+----------+------------------+-----------+--------+-------------+-------------------+---------+------------------+--------------------+--------------------+--------------------+--------------------+
 DATE|AGENDA_ITEM_NUMBER| ITEM_TYPE|DISTRICT| TITLE| VOTER NAME|VOTE CAST|FINAL ACTION TAKEN|AGENDA ITEM DESCR...| AGENDA_ID| VOTE_ID| [VOTE_ID]|
02/08/2017| 1| AGENDA| 13|Councilmember| Jennifer S. Gates| N/A| NO ACTION NEEDED| Call to Order| 020817__Special__1|020817__Special__...|[020817__Special_...|
02/08/2017| 1| AGENDA| 14|Councilmember| Philip T. Kingston| N/A| NO ACTION NEEDED| Call to Order| 020817__Special__1|020817__Special__...|[020817__Special_...|
02/08/2017| 1| AGENDA| 15| Mayor|Michael S. Rawlings| N/A| NO ACTION NEEDED| Call to Order| 020817__Special__1|020817__Special__...|[020817__Special_...|
02/08/2017| 1| AGENDA| 2|Councilmember| Adam Medrano| N/A| NO ACTION NEEDED| Call to Order| 020817__Special__1|020817__Special__1_2|[020817__Special_...|
02/08/2017| 1| AGENDA| 3|Councilmember| "Casey Thomas| II"| N/A| NO ACTION NEEDED| Call to Order| 020817__Special__1|[020817__Special_...|
02/08/2017| 1| AGENDA| 4|Councilmember|Carolyn King Arnold| N/A| NO ACTION NEEDED| Call to Order| 020817__Special__1|020817__Special__1_4|[020817__Special_...|
02/08/2017| 4|AGENDA DATE| 1|Councilmember| Scott Griggs| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[ and maintenance...|
02/08/2017| 4|AGENDA DATE| 10|Councilmember| B. Adam McGough| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[ and maintenance...|
02/08/2017| 4|AGENDA DATE| 11|Councilmember| Lee Kleinman| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[ and maintenance...|
02/08/2017| 4|AGENDA DATE| 12|Councilmember| Sandy Greyson| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[ and maintenance...|
02/08/2017| 4|AGENDA DATE| 13|Councilmember| Jennifer S. Gates| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[ and maintenance...|
02/08/2017| 4|AGENDA DATE| 14|Councilmember| Philip T. Kingston| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[ and maintenance...|
02/08/2017| 4|AGENDA DATE| 15| Mayor|Michael S. Rawlings| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[ and maintenance...|
02/08/2017| 4|AGENDA DATE| 2|Councilmember| Adam Medrano| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[ and maintenance...|
02/08/2017| 4|AGENDA DATE| 3|Councilmember| "Casey Thomas| II"| YES| APPROVED|"Authorize a thre...| calibration|[ calibration, a...|
02/08/2017| 4|AGENDA DATE| 4|Councilmember|Carolyn King Arnold| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[ and maintenance...|
02/08/2017| 4|AGENDA DATE| 5|Councilmember| Rickey D. Callahan| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[ and maintenance...|
01/11/2017| 1|AGENDA DATE| 13|Councilmember| Jennifer S. Gates| YES| APPROVED|"Approval of Minu...| 2016 City Counci...| 011117__1|[011117__1, 01111...|
04/25/2018| 1|AGENDA DATE| 12|Councilmember| Sandy Greyson| YES| APPROVED|"Approval of Minu...| 2018 City Counci...| 042518__1|[042518__1, 04251...|
+----------+------------------+-----------+--------+-------------+-------------------+---------+------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
votes_df = spark.read.csv("/FileStore/tables/DallasCouncilVotes_csv.gz", sep ='|')
valid_folders_df = votes_df.withColumnRenamed('_c0', 'folder')
split_df = split_df.withColumnRenamed('_c0', 'folder')
split_count = split_df.count()
joined_df = split_df.join(F.broadcast(valid_folders_df), "folder")
joined_df.show()
joined_count = joined_df.count()
print("Before: %d\nAfter: %d" % (split_count, joined_count))

+--------------------+--------+----------+------------------+-----------+--------+-------------+-------------------+---------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 folder|colcount| DATE|AGENDA_ITEM_NUMBER| ITEM_TYPE|DISTRICT| TITLE| VOTER_NAME|VOTE_CAST|FINAL_ACTION_TAKEN| AGENDA_ITEM_DESC| AGENDA_ID| VOTE_ID| split_cols| dog_list|
+--------------------+--------+----------+------------------+-----------+--------+-------------+-------------------+---------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
DATE,AGENDA_ITEM_...| 11| DATE|AGENDA_ITEM_NUMBER| ITEM_TYPE|DISTRICT| TITLE| VOTER NAME|VOTE CAST|FINAL ACTION TAKEN|AGENDA ITEM DESCR...| AGENDA_ID| VOTE_ID|[DATE, AGENDA_ITE...| [VOTE_ID]|
02/08/2017,1,AGEN...| 11|02/08/2017| 1| AGENDA| 13|Councilmember| Jennifer S. Gates| N/A| NO ACTION NEEDED| Call to Order| 020817__Special__1|020817__Special__...|[02/08/2017, 1, A...|[020817__Special_...|
02/08/2017,1,AGEN...| 11|02/08/2017| 1| AGENDA| 14|Councilmember| Philip T. Kingston| N/A| NO ACTION NEEDED| Call to Order| 020817__Special__1|020817__Special__...|[02/08/2017, 1, A...|[020817__Special_...|
02/08/2017,1,AGEN...| 11|02/08/2017| 1| AGENDA| 15| Mayor|Michael S. Rawlings| N/A| NO ACTION NEEDED| Call to Order| 020817__Special__1|020817__Special__...|[02/08/2017, 1, A...|[020817__Special_...|
02/08/2017,1,AGEN...| 11|02/08/2017| 1| AGENDA| 2|Councilmember| Adam Medrano| N/A| NO ACTION NEEDED| Call to Order| 020817__Special__1|020817__Special__1_2|[02/08/2017, 1, A...|[020817__Special_...|
02/08/2017,1,AGEN...| 12|02/08/2017| 1| AGENDA| 3|Councilmember| "Casey Thomas| II"| N/A| NO ACTION NEEDED| Call to Order| 020817__Special__1|[02/08/2017, 1, A...|[020817__Special_...|
02/08/2017,1,AGEN...| 11|02/08/2017| 1| AGENDA| 4|Councilmember|Carolyn King Arnold| N/A| NO ACTION NEEDED| Call to Order| 020817__Special__1|020817__Special__1_4|[02/08/2017, 1, A...|[020817__Special_...|
02/08/2017,4,AGEN...| 20|02/08/2017| 4|AGENDA DATE| 1|Councilmember| Scott Griggs| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[02/08/2017, 4, A...|[ and maintenance...|
02/08/2017,4,AGEN...| 20|02/08/2017| 4|AGENDA DATE| 10|Councilmember| B. Adam McGough| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[02/08/2017, 4, A...|[ and maintenance...|
02/08/2017,4,AGEN...| 20|02/08/2017| 4|AGENDA DATE| 11|Councilmember| Lee Kleinman| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[02/08/2017, 4, A...|[ and maintenance...|
02/08/2017,4,AGEN...| 20|02/08/2017| 4|AGENDA DATE| 12|Councilmember| Sandy Greyson| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[02/08/2017, 4, A...|[ and maintenance...|
02/08/2017,4,AGEN...| 20|02/08/2017| 4|AGENDA DATE| 13|Councilmember| Jennifer S. Gates| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[02/08/2017, 4, A...|[ and maintenance...|
02/08/2017,4,AGEN...| 20|02/08/2017| 4|AGENDA DATE| 14|Councilmember| Philip T. Kingston| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[02/08/2017, 4, A...|[ and maintenance...|
02/08/2017,4,AGEN...| 20|02/08/2017| 4|AGENDA DATE| 15| Mayor|Michael S. Rawlings| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[02/08/2017, 4, A...|[ and maintenance...|
02/08/2017,4,AGEN...| 20|02/08/2017| 4|AGENDA DATE| 2|Councilmember| Adam Medrano| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[02/08/2017, 4, A...|[ and maintenance...|
02/08/2017,4,AGEN...| 21|02/08/2017| 4|AGENDA DATE| 3|Councilmember| "Casey Thomas| II"| YES| APPROVED|"Authorize a thre...| calibration|[02/08/2017, 4, A...|[ calibration, a...|
02/08/2017,4,AGEN...| 20|02/08/2017| 4|AGENDA DATE| 4|Councilmember|Carolyn King Arnold| YES| APPROVED|"Authorize a thre...| calibration| and maintenance ...|[02/08/2017, 4

In [0]:
split_count = split_df.count() 
joined_count = joined_df.count()
invalid_df = split_df.join(F.broadcast(joined_df), 'folder', 'left_anti')
invalid_count = invalid_df.count()
print(" split_df:\t%d\n joined_df:\t%d\n invalid_df: \t%d" % (split_count, joined_count, invalid_count))

invalid_folder_count = invalid_df.select('folder').distinct().count()
print("%d distinct invalid folders found" % invalid_folder_count)

split_df:	44626
 joined_df:	44626
 invalid_df: 0
0 distinct invalid folders found

In [0]:
print(joined_df.select('dog_list').show(10, truncate=False))

DogType = StructType([
StructField("breed", StringType(), False),
StructField("start_x", IntegerType(), False),
StructField("start_y", IntegerType(), False),
StructField("end_x", IntegerType(), False),
StructField("end_y", IntegerType(), False)
])

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
dog_list |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
[VOTE_ID] |
[020817__Special__1_13] |
[020817__Special__1_14] |
[020817__Special__1_15] |
[020817__Special__1_2] |
[020817__Special__1, 020817__Special__1_3] |
[020817__Special__1_4] |
[ and maintenance services - Nicol Scales L.P. dba Nicol Scales & Measurement, single bid - Not to exceed $371, 205 - Financing: Current Funds ($134, 210), Water Utilities Current Funds ($179, 465), and Sanitation Current Funds ($57, 530) (subject to annual appropriations)", 020817__4, 020817__4_1] |
[ and maintenance services - Nicol Scales L.P. dba Nicol Scales & Measurement, single bid - Not to exceed $371, 205 - Financing: Current Funds ($134, 210), Water Utilities Current Funds ($179, 465), and Sanitation Current Funds ($57, 530) (subject to annual appropriations)", 020817__4, 020817__4_10]|
[ and maintenance services - Nicol Scales L.P. dba Nicol Scales & Measurement, single bid - Not to exceed $371, 205 - Financing: Current Funds ($134, 210), Water Utilities Current Funds ($179, 465), and Sanitation Current Funds ($57, 530) (subject to annual appropriations)", 020817__4, 020817__4_11]|
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
only showing top 10 rows

None

In [0]:
## dataset not availabe

In [0]:
## dataset not availabe